In [1]:
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
pygame.mixer.init()

In [3]:
height_screen = 480
width_screen = 640

In [4]:
# Create the screen

screen = pygame.display.set_mode([width_screen, height_screen])

# Background image

background = pygame.image.load('background.png')

In [5]:
# Title & Icon

pygame.display.set_caption('Breakout Game')
icon = pygame.image.load('universe.png')
pygame.display.set_icon(icon)

In [6]:
# Sounds

collision_sound = pygame.mixer.Sound('explosion.wav')

In [7]:
# Player

class Player(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)

        self.surf = pygame.image.load('bottom.png')
        self.size_x = 100
        self.player_pos_x = (width_screen/2)
        self.player_pos_y = height_screen - 40
        self.rect = self.surf.get_rect(center=(self.player_pos_x, self.player_pos_y))
        self.speed = 3


    def move_player(self):
        keys = pygame.key.get_pressed()

        if keys[pygame.K_LEFT] and self.rect.left > 0:
            self.rect.move_ip(-self.speed, 0)
        if keys[pygame.K_RIGHT] and self.rect.right < width_screen:
            self.rect.move_ip(self.speed, 0)
            
    def reset(self):
        self.player_pos_x = (width_screen/2)
        self.player_pos_y = height_screen - 40
        self.rect = self.surf.get_rect(center=(self.player_pos_x, self.player_pos_y))

player = Player()

In [8]:
# Ball

class Ball(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        
        self.size = 32
        self.surf = pygame.image.load('lua.png')
        self.ball_x = player.player_pos_x + (player.size_x/2) - (self.size/2)
        self.ball_y = player.player_pos_y - (self.size)
        self.rect = self.surf.get_rect(center=(self.ball_x, self.ball_y))
        self.speed = [2,2]
    
    def move_ball(self):
        self.rect = self.rect.move(self.speed)
        
        if self.rect.left < 0 or self.rect.right > width_screen:
            self.speed[0] = -self.speed[0]
        if self.rect.top < 0:
            self.speed[1] = -self.speed[1]
        if self.rect.bottom > height_screen:
            game_over('loser')
            
    def reset(self):
        self.ball_x = player.player_pos_x + (player.size_x/2) - (self.size/2)
        self.ball_y = player.player_pos_y - (self.size)
        self.rect = self.surf.get_rect(center=(self.ball_x, self.ball_y))
        self.speed = [2,2]

ball = Ball()

In [9]:
# Stars

class Star(pygame.sprite.Sprite):
    def __init__(self, x, y):
        pygame.sprite.Sprite.__init__(self)
        
        self.surf = pygame.image.load('star.png')
        self.rect = self.surf.get_rect(center=(x,y))
        
        
stars = pygame.sprite.Group()
  
def create_stars():
    star_size = 32
    num_row = 6
    star_x = star_size/2
    star_y = star_size/2
    while star_y < star_size * num_row:
        while star_x < width_screen:
            star = Star(star_x, star_y)
            stars.add(star)
            star_x += star_size
        star_x = (star_size/2)
        star_y += star_size
        
create_stars()

In [10]:
# Score

score_value = 0
font = pygame.font.Font('freesansbold.ttf',22)

def show_score(score_value):
    score = font.render(f'Score: {score_value}', True, (255,255,255))
    screen.blit(score, (10, 450))

In [11]:
# Game Over

def game_over(img):
    gameover_img = pygame.image.load(f'{img}.png')
    gameover_rect = gameover_img.get_rect(center=(width_screen/2, height_screen/2 - 50))
    screen.blit(gameover_img, gameover_rect)

    restart_img = pygame.image.load('restart.png')
    restart_rect = restart_img.get_rect(center=(width_screen/2, height_screen/2 + 130))
    screen.blit(restart_img, restart_rect)

    ball.speed = [0, 0]

    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    if click[0] == 1:
        if width_screen/2 - 32 < mouse[0] < width_screen/2 + 32  and height_screen/2 + 130 + 32 > mouse[1] > height_screen/2 + 130 - 32:
            stars.empty()
            create_stars()
            ball.reset()
            player.reset()

In [12]:
# Game Loop

running = True

while running:
    
    # Background
    screen.blit(background, (0, 0))
        
    # Player
    screen.blit(player.surf, player.rect)
    player.move_player()

    # Stars
    for star in stars:
        screen.blit(star.surf, star.rect)

    # Ball
    screen.blit(ball.surf, ball.rect)
    ball.move_ball()

    # Quit game
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Colision
    if ball.rect.colliderect(player.rect):
        ball.speed[1] = -ball.speed[1]

    collided_stars = pygame.sprite.spritecollide(ball, stars, True)
    if collided_stars:
        collision_sound.play()
        ball.speed[1] = -ball.speed[1]

    # Score
    for star in collided_stars:
        score_value += 1
    if len(stars) == 6 * 20:
        score_value = 0

    show_score(score_value)
    
    if len(stars) == 0:
        game_over('winner')

    pygame.display.update()


pygame.quit()